In [1]:
# mnist_cnn_keras

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
mnist = tf.keras.datasets.mnist
(x_train,y_train),(x_test,y_test) = mnist.load_data()

In [3]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


In [4]:
# one-hot 인코딩
nb_classes = 10 # 분류 class의 개수

Y_one_hot = tf.one_hot(y_train, nb_classes) # (60000,)
Y_one_hot.shape 

TensorShape([60000, 10])

In [5]:
# x를 실수형으로 변환
x_train = tf.cast(x_train,dtype=tf.float32)
x_test = tf.cast(x_test,dtype=tf.float32)
print(x_train.shape)

(60000, 28, 28)


In [6]:
# x의 shape을 4차원으로 변환
x_train = tf.reshape(x_train,[-1,28,28,1])
print(x_train.shape) # (60000, 28, 28, 1)
x_test = tf.reshape(x_test,[-1,28,28,1])
print(x_test.shape) # (10000, 28, 28, 1)
print(y_train.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)
(60000,)


In [8]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3),padding="same",activation='relu',
                               input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2),padding='same'),
        tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3),padding="same",activation='relu',
                               input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2),padding='same'),
        tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3),padding="same",activation='relu',
                               input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2),padding='same'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=10, activation='softmax')
    ]
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 7, 7, 128)         73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 4, 4, 128)        0

In [9]:
# https://yeomko.tistory.com/40
# Xavier Glorot Initialization : W(Weight) 값을 fan_in,fan_out를 사용하여 초기화하여 정확도 향상

# loss 종류
# mean_squared_error : 평균제곱 오차
# binary_crossentropy : 이진분류 오차
# categorical_crossentropy : 다중 분류 오차. one-hot encoding 클래스, [0.2, 0.3, 0.5] 와 같은 출력값과 실측값의 오차값을 계산한다.
# sparse_categorical_crossentropy: 다중 분류 오차. 위와 동일하지만 , integer type 클래스라는 것이 다르다.

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss = "sparse_categorical_crossentropy",
              metrics=['accuracy'])

In [10]:
# 학습
model.fit(x_train,y_train,epochs=25,batch_size=32,validation_split=0.25)

Epoch 1/25
1407/1407 [==============================] - 17s 12ms/step - loss: 0.2555 - accuracy: 0.9512 - val_loss: 0.0753 - val_accuracy: 0.9777
Epoch 2/25
1407/1407 [==============================] - 17s 12ms/step - loss: 0.0579 - accuracy: 0.9821 - val_loss: 0.0735 - val_accuracy: 0.9785
Epoch 3/25
1407/1407 [==============================] - 17s 12ms/step - loss: 0.0455 - accuracy: 0.9860 - val_loss: 0.0799 - val_accuracy: 0.9761
Epoch 4/25
1407/1407 [==============================] - 17s 12ms/step - loss: 0.0413 - accuracy: 0.9870 - val_loss: 0.0612 - val_accuracy: 0.9826
Epoch 5/25
1407/1407 [==============================] - 17s 12ms/step - loss: 0.0331 - accuracy: 0.9898 - val_loss: 0.0635 - val_accuracy: 0.9857
Epoch 6/25
1407/1407 [==============================] - 17s 12ms/step - loss: 0.0305 - accuracy: 0.9910 - val_loss: 0.0591 - val_accuracy: 0.9850
Epoch 7/25
1407/1407 [==============================] - 17s 12ms/step - loss: 0.0316 - accuracy: 0.9910 - val_loss: 0.0941 -

In [11]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 1s 4ms/step - loss: 0.2305 - accuracy: 0.9849


[0.23053781688213348, 0.9848999977111816]

In [13]:
preds = model.predict(x_test[:10])
print(tf.argmax(preds,axis=1).numpy())
print(y_test[:10])

1/1 [==============================] - 0s 17ms/step
[7 2 1 0 4 1 4 9 5 9]
[7 2 1 0 4 1 4 9 5 9]


In [7]:
### VGGNet (VGG-19) 스타일의 MNIST 분류 CNN 모델 
#--------------------------------------------
# ( Conv2D * 2개  --> MaxPool2D ) * 2회 : 4층
# ( Conv2D * 4개  --> MaxPool2D ) * 3회 : 12층
# Dense * 3개                           : 3층
#--------------------------------------------
#                                     총 19층
#--------------------------------------------
# 각 네트워크마다 필터의 수를 2배로 증가 시킨다 : 32-->64-->128-->256-->512

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3),padding='same',
                           activation='relu',input_shape=(28,28,1)),
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3),padding='same',
                           activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3),padding='same',
                           activation='relu'),
    tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3),padding='same',
                           activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(units=10, activation='softmax')
])

model.summary()

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss = "sparse_categorical_crossentropy",
              metrics=['accuracy'])
    

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 14, 14, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 14, 14, 128)       73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 14, 14, 256)       295168    
                                                        

In [8]:
# 학습
model.fit(x_train,y_train,epochs=25,batch_size=32,validation_split=0.25)

Epoch 1/25
1407/1407 [==============================] - 92s 65ms/step - loss: 0.6266 - accuracy: 0.8393 - val_loss: 0.1033 - val_accuracy: 0.9685
Epoch 2/25
1407/1407 [==============================] - 91s 65ms/step - loss: 0.1754 - accuracy: 0.9517 - val_loss: 0.0515 - val_accuracy: 0.9854
Epoch 3/25
1407/1407 [==============================] - 92s 65ms/step - loss: 0.1410 - accuracy: 0.9620 - val_loss: 0.0826 - val_accuracy: 0.9767
Epoch 4/25
1407/1407 [==============================] - 91s 65ms/step - loss: 0.1260 - accuracy: 0.9660 - val_loss: 0.0521 - val_accuracy: 0.9866
Epoch 5/25
1407/1407 [==============================] - 93s 66ms/step - loss: 0.1119 - accuracy: 0.9704 - val_loss: 0.0559 - val_accuracy: 0.9838
Epoch 6/25
1407/1407 [==============================] - 92s 66ms/step - loss: 0.1061 - accuracy: 0.9722 - val_loss: 0.0507 - val_accuracy: 0.9863
Epoch 7/25
1407/1407 [==============================] - 93s 66ms/step - loss: 0.1035 - accuracy: 0.9725 - val_loss: 0.0474 -